# Imports

In [8]:
from tqdm import tqdm
import joblib
import re
import spacy
import nltk
import pandas as pd

import sys
import json
#import stanza
#stanza.download('pt')
import numpy as np
# install and import amr utils
sys.path.append('../')
sys.path.append("../src/")
from features.nlp_preprocess import *
#!pip install ../amr-utils
from amr_utils.amr_readers import AMR_Reader

In [9]:
# folder = '../POS-tagger-portuguese-nltk/trained_POS_taggers/'
# tagger_nltk = joblib.load(folder+'POS_tagger_brill.pkl')

# Create dict annotation

In [10]:
path_lp = '../data/raw/little_prince.txt'
path_opisums = '../data/raw/opisums.txt'
path_news = '../data/raw/news.txt'
path_sci = '../data/raw/science.txt'

list_paths = [
    ('sci',path_sci),
    
    ('lp', path_lp),
    ('opisums',path_opisums),
    ('news',path_news),
]

In [11]:
def find_annotated_sentence(sentence, lines):
    
    result = []
    sentence_found = False

    for line in lines:
        if line.strip().startswith("# ::snt"):
            if sentence_found:
                # Stop collecting lines once we reach the next sentence
                break
            if sentence in line:
                # Start collecting lines once we find the target sentence
                sentence_found = True
        
        if sentence_found:
            result.append(line.strip())
    
    return "\n".join(result)

dict_path_alg = {
    'sci':"../AMR-Aligner/output/4aligner_science.txt.aligned",
    'lp': "../AMR-Aligner/output/4aligner_little_prince.txt.aligned", 
    'opisums':"../AMR-Aligner/output/4aligner_opisums.txt.aligned",
    'news':"../AMR-Aligner/output/4aligner_news_without_alignment.txt.aligned",
    
}

In [12]:
def aligments_aligner_to_dict(sentence):
    # Split the input sentence into lines
    lines = sentence.strip().split('\n')

    # Initialize the dictionaries
    node_text_dict = {}
    node_position_dict = {}

    # Process the lines to extract the required information
    for line in lines:
        if line.startswith('# ::node'):
            # Extract node information
            parts = line.split('\t')
            node_id = parts[1]
            node_text = parts[2]
            node_position = parts[3]
            
            # Populate the dictionaries
            node_text_dict[node_id] = node_text
            node_position_dict[node_id] = node_position

    return node_text_dict, node_position_dict

In [13]:
def convert_missmatch(input):
    
    if input == '"Brasil"':
        
        return '"Brazil"'
    else:
        return input

In [16]:
df_features = pd.DataFrame({
    'sentence_id': [],
    "corpus_name": [],
    'parent': [],
    'child': [],
    'parent_pos': [],
    'child_pos': [],
    'parent_ner': [],
    'child_ner': [],
    'dependency_role': [],
    'parent_position': [],
    'child_position': [],
    'label':[]
})

nlp_model_spacy = spacy.load("pt_core_news_lg")

list_dict_an = []

for corpus_name, path in list_paths:
    
    print(f"""##################################################
# Running: {corpus_name}
##################################################""")
        
    # usa reader de amr para fazer o parsing
    reader = AMR_Reader()
    amrs = reader.load(path, remove_wiki=True)
    
    # cria bloco de textos anotados
    with open(path, 'r') as file:
        str_corpus = file.read()
    blocos = str_corpus.split('\n\n')
    blocos = [bloco for bloco in blocos if bloco != ""]
        
    list_dicts = []
    # para cada sentenca, cria dict com os nós e vértices
    c = 0
    qtd_sents = len(amrs)
    for i,amr in tqdm(enumerate(amrs), total = qtd_sents):
        
        texto_anotado = blocos[i]
        dict_aligments = parse_alignment(texto_anotado)
        
        amr_string = amr.amr_string() # obtem string formatada amr        
        
        dict_annotation = amr_to_dict(amr) 
                
        if dict_aligments is None:
            
            snt = dict_annotation['snt']

            # le arquivo com aligments gerados pelo aligner
            with open(dict_path_alg[corpus_name], 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            # dado a sentenca atual, achar-la no arquivo com as sentencas anotadas pelo aligner
            annotated_sentence = find_annotated_sentence(snt,lines )         
            
            # extrai os aligments da sentenca anotada
            node_text_dict, node_position_dict = aligments_aligner_to_dict(annotated_sentence)
            
            # dicionario que mapeia os ids obtidos anteriormente e os id do aligner
            map_ids_nodes = {key_new:key_old for key_new, value_new in node_text_dict.items() for key_old, value_old in dict_annotation['nodes'].items() if convert_missmatch(value_old) == value_new}

            # cria dict alignments com os ids "certos"
            dict_aligments_unf = {map_ids_nodes.get(key):value for key, value in node_position_dict.items()}
            dict_aligments = dict_aligments_unf.copy()
            dict_aligments.pop(None, None)
            
            if None in dict_aligments_unf.keys():
                
                c+=1
            
                print(f"{c}/{qtd_sents} - node_text_dict:{node_text_dict}\nnode_position_dict:{node_position_dict}\ndict_annotation['nodes']:{dict_annotation['nodes']}\nmap_ids_nodes:{map_ids_nodes}\n")
            
            for key, value in dict_aligments_unf.items():
                
                start, end = value.split('-')
                
                start, end = int(start), int(end)
                
                dict_aligments.update({key: start})
                
            print(dict_aligments)
        
        dict_annotation.update({'dict_aligments': dict_aligments})
        dict_annotation.update({'corpus_name': corpus_name})
        list_dicts.append(dict_annotation)
        
    print('Anotando texto ...')    
    for dict_an in tqdm(list_dicts):
        
        tokens = dict_an['tokens_nltk']
        
        if tokens == []:
            tokens = dict_an['tok pt']
        
        # tok_pos = pos_tagger_nltk(
        # tokenized_sentence = tokens,
        # tagger_nltk = tagger_nltk
        # )
          
        dict_spacy = extract_feat_spacy(
            text = dict_an['snt'],
            nlp_model = nlp_model_spacy)
        
                
        dict_an.update(dict_spacy)
        #dict_an.update({'tok pos': tok_pos})
        dict_an.update({"corpus_name": corpus_name})
        
        list_dict_an.append(dict_an)

    print()
    
with open('../data/processed/annotated_text_alignments.json', 'w') as f:
    json.dump(list_dict_an, f)

cannot deinvert attribute: ('s2', ':op2-of', 'and')


##################################################
# Running: sci
##################################################
[amr] Loading AMRs from file: ../data/raw/science.txt


100%|██████████| 160/160 [00:00<00:00, 1156.86it/s]


{'1': 1, '1.1': 5, '1.1.1': 6}
{'1': 1, '1.1': 0}
{'1': 0, '1.1': 3, '1.1.1': 6}
{'1': 6, '1.1.1.1': 2, '1.2': 5, '1.2.1': 4, '1.3': 7}
{'1': 5, '1.1.2.1': 7, '1.2': 2, '1.2.1': 1, '1.2.1.1': 11}
{'1': 4, '1.1': 2, '1.1.1': 1, '1.1.2.2.1': 6, '1.2': 3}
{'1': 3, '1.1': 6, '1.1.1': 7, '1.1.1.1': 4}
{'1': 2, '1.1': 1, '1.2': 7, '1.2.1': 5}
{'1': 3, '1.1': 9, '1.1.1': 8}
{'1': 3, '1.1': 5, '1.1.1': 7, '1.2': 0, '1.2.1': 1}
{'1': 2, '1.1': 3, '1.1.1': 4, '1.1.1.1': 7}
{'1': 5, '1.2': 1, '1.1.1': 4}
{'1': 0, '1.1': 8, '1.1.1': 7, '1.1.1.1': 10}
{'1': 2, '1.1': 3, '1.1.1': 5, '1.1.1.1': 7, '1.1.2': 1, '1.1.2.1': 0}
{'1': 4, '1.1': 6, '1.2': 9}
{'1': 2, '1.1': 4, '1.1.1': 9, '1.2': 8}
{'1': 3, '1.1.2.1': 5}
1/160 - node_text_dict:{'0': 'falar-01', '0.1': 'person', '0.1.0': '-', '0.1.0.0.0': 'name', '0.1.0.0.0.0': '"Araújo"', '0.1.0.0.0.0.0': 'querer-01'}
node_position_dict:{'0': '3-4', '0.1': '2-3', '0.1.0': '1-2', '0.1.0.0.0': '5-6', '0.1.0.0.0.0': '7-8', '0.1.0.0.0.0.0': '4-5'}
dict_annotati

100%|██████████| 160/160 [00:00<00:00, 180.11it/s]


...

 65%|██████▍   | 564/870 [00:00<00:00, 5626.55it/s]

{'1': 4, '1.1': 1, '1.2': 2, '1.3': 3}
{'1': 1, '1.1': 2, '1.2': 0}
{'1': 0}
{'1': 0, '1.1': 4, '1.1.2': 3}
{'1': 2}
{'1': 2, '1.1': 0}
{'1': 1, '1.1': 3}
{'1': 3}
{'1': 2, '1.1': 1, '1.2': 0}
{'1': 0}
{'1': 3, '1.1': 1}
{'1': 2, '1.1': 4}
{'1': 2, '1.2': 4}
{'1': 2, '1.1': 4, '1.2': 3}
{'1': 4}
{'1': 2}
{'1': 2, '1.1': 0}
{'1': 3, '1.1': 4, '1.2': 0}
{'1': 0, '1.1': 2, '1.2': 1, '1.2.1': 4}
{'1': 2}
{'1': 0}
{'1': 2}
{'1': 4, '1.1': 1}
{'1': 1, '1.1': 0, '1.2': 3}
{'1': 4, '1.1': 2}
{'1': 3, '1.1': 1}
{'1': 0}
{'1': 2, '1.1': 0}
{'1': 3, '1.1': 0}
{}
{'1': 1, '1.1': 0, '1.2': 4, '1.3': 3}
{'1': 1, '1.1': 2, '1.2': 4, '1.3': 0}
{'1': 0, '1.1': 1, '1.1.1': 2, '1.1.1.1': 3}
{'1': 4}
{}
{'1': 1, '1.1.1.1': 4}
{'1': 2}
{'1': 2, '1.1': 4}
{'1': 1, '1.1': 0, '1.2': 2}
{'1': 3}
{'1': 3, '1.1': 2, '1.2': 4}
{'1': 1}
{'1': 3, '1.3': 0, '1.3.1': 2, '1.1.2': 4}
{'1': 0, '1.1': 2, '1.1.1': 4}
{'1': 1, '1.1': 3, '1.2': 0}
{'1': 2, '1.1': 1, '1.1.1': 4}
{'1': 3}
{'1': 2}
{'1': 0, '1.1': 1, '1.2': 4,

100%|██████████| 870/870 [00:00<00:00, 2597.31it/s]


{'1': 1, '1.2.1': 2}
{'1': 12, '1.1': 5, '1.2.1.1': 6, '1.1.1.1': 7}
{'1': 0, '1.1': 2}
{'1': 4, '1.1': 2, '1.1.1': 7, '1.2': 8, '1.2.1': 9}
{'1': 0, '1.1.1': 6}
{'1': 3, '1.1': 6, '1.1.1': 8, '1.2': 5}
{'1': 0, '1.1': 1, '1.1.1': 3, '1.1.2': 6}
Anotando texto ...


100%|██████████| 870/870 [00:04<00:00, 200.60it/s]
